# GMAI-VL 의료 영상 분석

> **Purpose:** GMAI-VL(General Medical AI Vision-Language) 모델을 활용하여
> 의료 영상(CT, MRI, X-ray)과 임상 텍스트를 통합 분석합니다.
>
> **모델 사양:**
> - 구조: Vision Encoder (ViT-L/14) + Text Encoder (BERT-base) + Cross-Attention
> - 학습 데이터: 550만 개 이미지-텍스트 쌍 (PubMed, Radiopaedia)
> - 강점: 다양한 의료 영상에서의 일반적 병변 탐지 및 해부학적 구조 인식

## 1. 라이브러리 및 모델 로드

In [ ]:

import torch
from PIL import Image
from pyspark.sql import functions as F

# MLflow 모델 로드 (Model Registry에서)
# model = mlflow.pytorch.load_model("models:/gmai-vl/Production")

# 또는 HuggingFace에서 직접 로드
# from transformers import AutoModel, AutoTokenizer
# model = AutoModel.from_pretrained("GMAI-VL-model-path")
# tokenizer = AutoTokenizer.from_pretrained("GMAI-VL-model-path")

print("🔧 GMAI-VL 모델 설정 준비")
print("   → MLflow Model Registry 또는 HuggingFace에서 로드")

## 2. 영상 전처리 파이프라인

In [ ]:

def preprocess_medical_image(image_path, target_size=(224, 224)):
    """
    의료 영상을 GMAI-VL 입력 형태로 전처리합니다.
    
    Args:
        image_path: ADLS Gen2 내 이미지 경로
        target_size: 모델 입력 크기
    
    Returns:
        torch.Tensor: 전처리된 이미지 텐서
    """
    try:
        img = Image.open(image_path).convert("RGB")
        img = img.resize(target_size)
        
        # 정규화 (ImageNet 통계량)
        import torchvision.transforms as transforms
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])
        
        return transform(img).unsqueeze(0)
    except Exception as e:
        print(f"⚠️ 이미지 전처리 실패: {e}")
        return None

print("✅ 영상 전처리 함수 준비 완료")

## 3. GMAI-VL 추론

In [ ]:

def gmai_vl_inference(image_tensor, clinical_context):
    """
    GMAI-VL 모델로 멀티모달 추론을 수행합니다.
    
    Args:
        image_tensor: 전처리된 이미지 텐서
        clinical_context: 환자 바이탈/증상 텍스트
    
    Returns:
        dict: {
            "findings": str,           # 발견 사항
            "abnormality_bbox": list,   # 병변 위치 (bounding box)
            "confidence": float         # 신뢰도
        }
    """
    # 모델 추론 (실제 구현)
    # with torch.no_grad():
    #     outputs = model(image_tensor, clinical_context)
    #     findings = outputs["text"]
    #     bbox = outputs["bbox"]
    #     confidence = outputs["confidence"]
    
    # 시뮬레이션 결과 (모델 로드 전 테스트용)
    result = {
        "findings": "Right middle lobe consolidation identified. No pleural effusion observed.",
        "abnormality_bbox": [120, 80, 200, 160],  # [x1, y1, x2, y2]
        "confidence": 0.89,
        "modality": "CT",
        "model_version": "GMAI-VL-v1.0"
    }
    
    return result

print("✅ GMAI-VL 추론 함수 준비 완료")

## 4. 배치 영상 분석

In [ ]:

def batch_image_analysis(image_paths, clinical_contexts):
    """
    다수의 의료 영상에 대해 GMAI-VL 배치 분석을 수행합니다.
    
    결과는 Gold Layer에 저장되어 OpenAI 추론의 입력으로 사용됩니다.
    """
    results = []
    
    for img_path, context in zip(image_paths, clinical_contexts):
        img_tensor = preprocess_medical_image(img_path)
        
        if img_tensor is not None:
            result = gmai_vl_inference(img_tensor, context)
            result["image_path"] = img_path
            results.append(result)
    
    if results:
        df_results = spark.createDataFrame(results)
        df_results = df_results.withColumn("analyzed_at", F.current_timestamp())
        
        # Gold Layer에 저장
        df_results.write \
            .format("delta") \
            .mode("append") \
            .saveAsTable("gold.image_analysis_results")
        
        print(f"✅ 영상 분석 완료: {len(results)}건")
    
    return results

print("✅ 배치 영상 분석 함수 준비 완료")
print("→ 50_MLflow_Experiment_Setup.py에서 실험 트래킹 연동")